In [ ]:
pip install face_recognition

In [ ]:
from tensorflow import keras

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Concatenate
from tensorflow.keras.layers import Flatten, Dropout, Dense, LeakyReLU, AveragePooling2D
from tensorflow.keras import Model
class model():
    def __init__(self,trained = False, input_shape = (299,299,3)):
        if not trained:
            self.model2 = self.make_model(input_shape)

    def make_model(self,input_shape):
        print("Tlqkf")
        model_input = Input(shape = input_shape)
      
        x = Conv2D(32,(3,3),strides=2)(model_input)
        x = Conv2D(64,(3,3))(x) 

        x = self.inception_layer(16,32,32,16)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2),padding='same')(x)

        x = self.inception_layer(8,16,16,16)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2),padding='same')(x)

        x = Conv2D(16,(5,5),padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2,2),padding='same')(x)
        
        x = Flatten()(x)
        x = Dropout(0.5)(x)

        x = Dense(32)(x)
        x = LeakyReLU(alpha = 0.2)(x)
        x = Dropout(0.5)(x)
        
        y = Dense(1, activation = 'sigmoid')(x)

        return  Model(inputs = model_input, outputs = y)



        
    def inception_layer(self,n1,n2,n3,n4):
        def inception(x):
            x1 = Conv2D(n1, (1,1), padding='same', activation = 'relu')(x)

            x2 = Conv2D(n2, (1,1), padding='same', activation = 'relu')(x)
            x2 = Conv2D(n2, (3,3), padding='same', activation = 'relu')(x2)

            x3 = Conv2D(n3, (1,1), padding='same', activation = 'relu')(x)
            x3 = Conv2D(n3, (5,5), padding='same', activation = 'relu')(x3)

            x4 = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding = 'same')(x)
            x4 = Conv2D(n4, (1,1), padding='same', activation = 'relu')(x4)
            output = Concatenate()([x1,x2,x3,x4])
            return output
        return inception


In [ ]:
from IPython.display import display
import numpy as np
import os
import glob
import pandas as pd
from PIL import Image, ImageDraw
import cv2 as cv
from tensorflow import keras
import random
import face_recognition

class Data(keras.utils.Sequence):
    def __init__(self,path, shuffle = False, batch_size = 32, num_frame = 10, get_img = False, mode = 'test'):
        super(Data,self).__init__()
        self.path = path
        self.batch_size = batch_size
        self.num_frame = num_frame
        self.shuffle = shuffle
        self.mode = mode
        if get_img:
            if mode =='train':
                self.read_json(self.path)
            else:
                self.test(self.path)
        else:
            self.read_folder(self.path)

    def __len__(self):
        return int(len(self.labels) / self.batch_size)

    def __getitem__(self, index):
        files = self.data[self.batch_size * index : self.batch_size * (index+1)]
        x = self.get_images(files)
        y = self.labels[self.batch_size * index : self.batch_size * (index+1)]
        return np.array(x), np.array(y)

    def on_epoch_end(self):
      self.indexes = np.arange(len(self.data))

      if self.shuffle:
        np.random.shuffle(self.indexes)

    def test(self,path):
        data = glob.glob(path+"*.mp4")
        labels = ['test' for x in data]
        n=0
        path = path+"images/"
        for i in range(len(data)):
            print(i)
            label = labels[i]
            faces = self.extractor(data[i],labels)
            for face in faces:
                f_name = path + str(n).zfill(4) + label + ".jpg"
                n = n + 1
                cv.imwrite(f_name, cv.cvtColor(face,cv.COLOR_RGB2BGR))
                
    def read_json(self,path):
        json = glob.glob(path+"*.json")[0]
        json = pd.read_json(json)
        files = list(json.keys())
        data = [path + x for x in files]
        labels = list(json.T['label'])
        self.data, self.labels = [], []
        n = 0
        path = path + "images/"
        if not (os.path.isdir(path[:-1])):
            os.makedirs(path[:-1])
        for i in range(len(data)):
            print(i)
            label = labels[i]
            faces = self.extractor(data[i],labels)
            for face in faces:
                f_name = path + str(n).zfill(4) + label + ".jpg"
                n = n + 1
                cv.imwrite(f_name, cv.cvtColor(face,cv.COLOR_RGB2BGR))
                self.data.append(f_name)
                self.labels.append(label)

    def get_images(self,files):
        x = []
        for file in files:
            image = keras.preprocessing.image.load_img(path=file)
            img = keras.preprocessing.image.img_to_array(image)
            x.append(img)
        return x

    def extractor(self, file, label):
        v_cap = cv.VideoCapture(file)
        v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
        faces = []
        sample = np.linspace(0, v_len - 1, self.num_frame).astype(int)

        for i in range(v_len):
            success = v_cap.grab()
            if i in sample:
                success, image = v_cap.retrieve()
                if not success:
                    break
                image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
                shape = image.shape
                location = face_recognition.face_locations(image)
                if location:
                    t, r, b, l = location[0]
                else:
                    continue
                t, r, b, l = max(0,t-40), min(shape[1], r+40), min(shape[0],b+40), max(0,l-40)
                face = image[t:b, l:r]
                face = cv.resize(face,dsize=(299,299))
                faces.append(face)
        return faces

    def read_folder(self,path):
        self.data = glob.glob(path+"*.jpg")
        self.labels = []
        size = len(self.data)
        if self.mode == 'train':
          self.data = self.data[:int(size*0.8)]
        elif self.mode == 'validation':
          self.data = self.data[int(size*0.8):]
        if self.shuffle:
            random.shuffle(self.data)
        for data in self.data:
            if 'FAKE' in data:
                self.labels.append(1)
            else:
                self.labels.append(0)


In [ ]:
t_set = Data(path='/content/drive/My Drive/dataset/train_sample_videos/images/',shuffle=True,get_img=False,mode='train')
v_set = Data(path='/content/drive/My Drive/dataset/train_sample_videos/images/',shuffle=False,get_img=False,mode='validation')
m = model()
inception_model = m.model2

callback_list = [
        keras.callbacks.ModelCheckpoint(
        filepath="/content/drive/My Drive/dataset/Saved_Model2.h5",
        monitor='val_acc',
        save_best_only=True
    )
]
inception_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['acc'])
inception_model.fit_generator(generator = asdf,steps_per_epoch=len(asdf),validation_data=qwert,
                epochs = 20,callbacks=callback_list)

Tlqkf
Epoch 1/20
89/89 [==============================] - 1330s 15s/step - loss: 0.7150 - acc: 0.7275 - val_loss: 0.8911 - val_acc: 0.8011
Epoch 2/20
89/89 [==============================] - 16s 182ms/step - loss: 0.5750 - acc: 0.7535 - val_loss: 0.8383 - val_acc: 0.6875
Epoch 3/20
89/89 [==============================] - 17s 189ms/step - loss: 0.5240 - acc: 0.7746 - val_loss: 0.4902 - val_acc: 0.7955
Epoch 4/20
89/89 [==============================] - 17s 194ms/step - loss: 0.5073 - acc: 0.7767 - val_loss: 0.8434 - val_acc: 0.8338
Epoch 5/20
89/89 [==============================] - 16s 183ms/step - loss: 0.4938 - acc: 0.7844 - val_loss: 1.4277 - val_acc: 0.6065
Epoch 6/20
89/89 [==============================] - 16s 181ms/step - loss: 0.4982 - acc: 0.7960 - val_loss: 0.7325 - val_acc: 0.8224
Epoch 7/20
89/89 [==============================] - 16s 182ms/step - loss: 0.4284 - acc: 0.8072 - val_loss: 0.6498 - val_acc: 0.7102
Epoch 8/20
89/89 [==============================] - 16s 185ms/s

In [ ]:
indeption_model.load_weights("/content/drive/My Drive/dataset/Saved_Model2.h5")